# Seminarska naloga iz Podatkovnega Rudarjenja 2020/2021

### Odločili smo se za nalogo: ukradena vozila.

### <font color='orange'>In progress [ &#9874; ] </font>&nbsp; <font color='red'>Declined [ ✗ ]</font> &nbsp;<font color='green'>Finished [ ✓ ]</font>

* <font color='green'>Priprava osnovnega parserja [ ✓ ]</font>   <br/>
* <font color='green'>Priprava tabele z kraticami in mesti registracijskih [ ✓ ]</font>   <br/>
* <font color='green'>Pridobi možne podatke s spletne strani avtolog.si [ ✓ ]</font>   <br/>
* <font color='orange'>v katerem mestu je največ kraj( glede na tablico MB,LJ,..) [ &#9874; ]</font>  <br/>
* <font color='orange'>katera znamka je najbloj kradena (lahko tudi glede na mesto) [ &#9874; ]</font>  <br/>
* <font color='orange'>kateri model je najbolj kraden (-||-) [ &#9874; ]</font>  <br/>
* <font color='orange'>ali je večina ukradenih starejših/novejših od X leta [ &#9874; ]</font>  <br/>
* <font color='orange'>ali so ukredena vozila dražja/cenejša od X vrednosti [ &#9874; ]</font>  <br/>
* <font color='orange'>koliko vozil je bilo ukradenih za vsako leto [ &#9874; ]</font>  <br/>
* <font color='orange'>katera barva je najbolj kradena [ &#9874; ]</font>  <br/>
* <font color='orange'>Ali so cena avtomobila in kraja avtomobila povezana [ &#9874; ]</font>  <br/>
* <font color='orange'>Verjetnost, da ti ukradejo avto če si iz mesta X (st_Ukradenih_avtov_mesto / prebivalstvo_mesto) [ &#9874; ]</font>  <br/>
* <font color='orange'>Kraja po starosti vozila [ &#9874; ]</font>  <br/>



In [1]:
import pandas as pd

In [78]:
#import sys
#!{sys.executable} -m pip install beautifulsoup4

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached soupsieve-2.2.1-py3-none-any.whl (33 kB)


In [10]:
vozila = pd.read_csv("ukrvozila.csv", sep=";")
kMesta = pd.read_csv("kraticeMesta.txt", sep=";")

In [11]:
vozila['znamka'].isnull().values.any() #true
vozila['tip'].isnull().values.any() #true
vozila['registrska'].isnull().values.any() #true
vozila['sasija'].isnull().values.any() #false
vozila['barva'].isnull().values.any() #true
vozila['datumodvzema'].isnull().values.any() #false

False

In [12]:
vozila['znamka'] = vozila['znamka'].fillna("")
vozila['tip'] = vozila['tip'].fillna("")
vozila['registrska'] = vozila['registrska'].fillna("")
vozila['barva'] = vozila['barva'].fillna("")

In [76]:
#vozila["registrska"][vozila["registrska"].str.contains("^I")]

5108     INSQ5005
15136      IEH257
Name: registrska, dtype: object

## Parser

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
import re

In [18]:
def getWebsiteData(sasija):
    
    url="http://avtolog.si/search/"+sasija+"/"
    html_content = requests.get(url).text
    parsed_html = BeautifulSoup(html_content,'html.parser')
    
    ## Get table data
    allTables = parsed_html.find_all("div", attrs={"class": "pane"})
    
    #Take the usefull data: Podatki o vozilu; Podatki o lastniku.
    d1 = allTables[0]
    d3 = allTables[2]

    t1 = d1.findAll("table")[0]
    t_headers = []
    for th in t1.find_all("th"):
        # remove any newlines and extra spaces from left and right
        t_headers.append(th.text.replace('\n', ' ').strip())
    #print(t_headers)
    table_data = []
    for td in t1.find_all("td"):
        a = re.sub("\n +"," ", td.text.strip())
        tdt = re.sub(" +"," ", a)
        table_data.append(tdt)
    sz = [x for x in zip(t_headers, table_data)]    
    if(d3.findAll("table") != []):
        t3 = d3.findAll("table")[0]
        tds = t3.find_all("td")[-3]
        a = re.sub("\n +"," ", tds.text.strip())
        sz.append(('Status/starost',re.sub(" +"," ", a)))
    s = ("sasija", sasija)
    sz.insert(0,s)
    return sz

In [9]:
#getWebsiteData("SWNH30000A0002248")

[('sasija', 'SWNH30000A0002248'), ('Znamka vozila', 'NIEWIADOW H3000 20K 5220'), ('Leto izdelave', '2011'), ('Število lastnikov', '1'), ('Barva', 'kovinski - SIVA - SVETLA'), ('Vrsta goriva', 'Ni goriva'), ('Oznaka motorja', 'None'), ('Moč', '-1 kW'), ('Prostornina', '-1 ccm'), ('EURO oznaka', 'ni podatka'), ('Prva registracija', '2011-07-26'), ('Prve registracija (SI)', '2011-07-26'), ('Status/starost', 'Pravna oseba')]


In [17]:
V0_2000 = vozila["sasija"][0:2000] #done
V2000_4000 = vozila["sasija"][2000:4000] #done
V4000_6000 = vozila["sasija"][4000:6000] #done
V6000_8000 = vozila["sasija"][6000:8000] #done
V8000_10000 = vozila["sasija"][8000:10000] #done
V10000_12000 = vozila["sasija"][10000:12000] #done
V12000_14000 = vozila["sasija"][12000:14000] #done
V14000_16000 = vozila["sasija"][14000:16000]
V16000_konc = vozila["sasija"][16000:]

In [21]:
#print(len(V0_2000) + len(V2000_4000) + len(V4000_6000) + len(V6000_8000) + len(V8000_10000) + len(V10000_12000) + len(V12000_14000) + len(V14000_16000) + len(V16000_konc))

16927


In [72]:
#Done: 6000 - errors
#V0_2000, 2000-4000, 4000-6000, 6000-8000, 8000-10000, 10000-12000
def getDataFromRange(array):
    c = 0;
    dataArray = []
    errorArray = []
    for x in array:
        try:
            podatki = getWebsiteData(x)
            dataArray.append(podatki)
        except:
            errorArray.append(c)
        c+=1
    return (dataArray, errorArray)


In [151]:
#Zaženi na vsake nekaj časa, da ne preobremenimo strežnika.
#dataArray, errorArray = getDataFromRange(V16000_konc)


In [152]:
#Preveri ali so podakti pravilno zapisani
#dataArray

[[('sasija', '863428'),
  ('Znamka vozila', 'None None'),
  ('Leto izdelave', 'None'),
  ('Število lastnikov', '1'),
  ('Barva', 'None'),
  ('Vrsta goriva', 'None'),
  ('Oznaka motorja', 'None'),
  ('Moč', '/ kW'),
  ('Prostornina', '/ ccm'),
  ('EURO oznaka', 'ni podatka'),
  ('Prva registracija', 'None'),
  ('Prve registracija (SI)', 'None')],
 [('sasija', 'ZAPM0800000004313'),
  ('Znamka vozila', 'PIAGGIO None'),
  ('Leto izdelave', 'None'),
  ('Število lastnikov', '2'),
  ('Barva', 'None'),
  ('Vrsta goriva', 'None'),
  ('Oznaka motorja', 'None'),
  ('Moč', '/ kW'),
  ('Prostornina', '/ ccm'),
  ('EURO oznaka', 'ni podatka'),
  ('Prva registracija', '1998-06-01'),
  ('Prve registracija (SI)', 'None UVOŽEN'),
  ('Status/starost', '57 let')],
 [('sasija', 'WDB2011221A124484'),
  ('Znamka vozila', 'MERCEDES-BENZ None'),
  ('Leto izdelave', 'None'),
  ('Število lastnikov', '1'),
  ('Barva', 'None'),
  ('Vrsta goriva', 'None'),
  ('Oznaka motorja', 'None'),
  ('Moč', '/ kW'),
  ('Prosto

In [153]:
#counter = []
#for x in dataArray:
#    counter.append(len(x))

In [154]:
#Preveri ali imajo vsi najmanj 12 in največ 13 vnosov.
#set(counter)

{12, 13}

In [155]:
#errorArray

[18,
 33,
 88,
 113,
 139,
 215,
 263,
 385,
 416,
 417,
 460,
 485,
 538,
 575,
 660,
 795,
 799,
 812,
 884]

In [156]:
#vozila.iloc[16000+18]

znamka                     NISSAN
tip                         SUNNY
registrska                LJS953U
sasija          JN1BCAN14UO761295
barva                     SREBRNA
datumodvzema           25.07.2001
Name: 16018, dtype: object

In [99]:
def addTotxtError(array, zacetna):
    file = open('napake.txt', 'a')
    for x in array:
        er = vozila["sasija"].iloc[x+zacetna]
        file.write(er + "\n")
    file.close()


In [62]:
import csv
def firstCsv():
    with open('normal.csv', mode='a', newline='') as voz_file:
        voz_writer = csv.writer(voz_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        headers = []
        for x in dataArray[0]:
            headers.append(x[0])
        voz_writer.writerow(headers)
        voz_file.close()

def addTocsvNormal(array):
     with open('normal.csv', mode='a', newline='') as voz_file:
        voz_writer = csv.writer(voz_file, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for x in array:
            podatki = [y[1] for y in x]
            if len(podatki) == 12:
                podatki.append("BREZ")
            voz_writer.writerow(podatki)    
        voz_file.close()

In [157]:
#Ko dobiš nove podatke zapiši vse errorje v datoteko.
#addTotxtError(errorArray, 16000)

In [63]:
#Zaženi samo prvič, da se ustvarijo headerji
#firstCsv()

In [158]:
#Vsakič ko dobiš nove podatke, zaženi.
#addTocsvNormal(dataArray)

# Obdelava podatkov | statistika podatkov

In [ ]:
##Pridobi združeno tabelo po šasiji

#merge = vozila.merge(avtolog)